In [1]:
import wilds
from wilds.datasets.waterbirds_dataset import WaterbirdsDataset
from torch.utils.data import Subset
import numpy as np

In [59]:
def majority_only_waterbirds_dataset(dataset:WaterbirdsDataset) -> WaterbirdsDataset:
    train_split_mask = dataset.split_array == dataset.split_dict["train"]
    train_split_idx = np.where(train_split_mask)[0]

    train_metadata_array = dataset.metadata_array[train_split_mask,:]

    #(train_metadata_array[:,0].numpy() == train_metadata_array[:,1].numpy()).mean()
    spurious = (train_metadata_array[:,0].numpy() == train_metadata_array[:,1].numpy())

    non_spurious_train_split_idx = train_split_idx[~spurious]

    dataset.split_array[non_spurious_train_split_idx] = -1

    return dataset

In [2]:
majority_only = True
dataset = WaterbirdsDataset(root_dir="datasets")
if majority_only:
    dataset = majority_only_waterbirds_dataset(dataset)
data_train = dataset.get_subset("train")
data_test  = dataset.get_subset("test")
data_valid = dataset.get_subset("val") 


NameError: name 'majority_only_waterbirds_dataset' is not defined

In [3]:
dataset.metadata_fields.index('background')

0

In [9]:
def f():
    for i in range(3):
        yield {"x":i , "y":i+1}

for x in f():
    print(x)

{'x': 0, 'y': 1}
{'x': 1, 'y': 2}
{'x': 2, 'y': 3}


In [2]:
import torch 
saved_data_train_path = "./datasets/office_home_train.pt"
data_train = torch.load(saved_data_train_path)


In [19]:
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import random

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


g = torch.Generator()
g.manual_seed(42)

test_dl = DataLoader(data_train, batch_size=512, worker_init_fn=seed_worker, generator=g,shuffle=False)


In [22]:
for batch in test_dl:
    print("hello")

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


In [33]:
data_test.tensors[0].size()

torch.Size([4357, 3, 90, 90])

In [48]:
from torch import default_generator, randperm
indices = randperm(4357, generator=torch.Generator().manual_seed(42)).tolist()

data_test.tensors[0][indices[1900:]].size()

torch.Size([2457, 3, 90, 90])

In [32]:
torch.cat((data_train.tensors[1],data_test.tensors[1])).size()

torch.Size([13161])

In [58]:
import numpy as np
import torch
np.sum(tensor([ True,  True,  True, False, False, False]))


NameError: name 'tensor' is not defined

In [25]:
def check_stats(data):

    spurious = (data.metadata_array[:,0].numpy() == data.metadata_array[:,1].numpy())
    confounder_strength = spurious.mean()

    print(f"Confounder_strength = {confounder_strength:.3f}")
    
    return confounder_strength



In [37]:
def majority_only(data):

    spurious = (data.metadata_array[:,0].numpy() == data.metadata_array[:,1].numpy())

    spurious_idx= np.arange(len(data))[spurious]

    return data[spurious_idx]
